In [ ]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")

for dataset in [ 'caltech101', ]:  
                # 'flowers-102', 'kvasir-dataset', 'uc-merced-land-use-dataset'
                # 'stanford_cars',
                # 'caltech101', 'dtd', 'eurosat' ,
                # 'fgvc-aircraft-2013b', 'oxford-iiit-pet' ]:

    print(dataset)
    
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N = load_data(False, config, common_corruption=False)


/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-20 16:40:55,785	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-20 16:40:55,893	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-20 16:40:55,987	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


caltech101


In [2]:
testloader = DataLoader(test_dataset, batch_size=2, ) 

for x, y in testloader:
    print(x.shape)
    print(y)
    break

torch.Size([2, 3, 224, 224])
tensor([ 0, 10])


In [ ]:
### don't pay attention to this cell

# EuroSAT: seulement 8 runs on complete et tous les autres OOM
# Caltech: 26 runs ok, 
# Stanford Cars: 34 ok
# Aircraft: 26 ok
# DTD : 26 ok
# Oxford pet: 26 ok
# Flowers: 26 ok

stanford_cars = 6922 + 1222 + 8041 
caltech101 = 5899 + 1042 + 1736 
dtd = 1880 + 1880 + 1880 
eurosat = 18360 + 3240 + 5400 
fgvcaircraft2013b =  3334 + 3333 + 3333 
flowers102 = 1020 + 1020 + 6149 
oxfordiiitpet = 3128 + 552 + 3669 

print(eurosat, caltech101, dtd, fgvcaircraft2013b, flowers102, oxfordiiitpet, stanford_cars)

from PIL import Image
import torchvision.datasets as datasets

# Initialize the Caltech101 dataset without transformations
caltech101_sample = datasets.Caltech101(root='./data', download=False)
print(len(caltech101_sample))
for i in range( len(caltech101_sample) ) :
    image, label = caltech101_sample[i]

    # Check the image mode
    # print(f"Image Mode: {image.mode}")  # Expected: 'RGB'
    if image.mode != 'RGB':
        print('issue')